# Notebook 5: Evaluation / Evaluación

In this hackathon, you will use this notebook to evaluate the performance of your machine learning pipeline. The notebook is designed to assess the entire pipeline, from data preprocessing to making predictions, ensuring that your solution is practical, efficient, and accurate.

<hr style="border:2px solid gray">


En este hackathon, utilizarás este Jupyter notebook para evaluar el rendimiento de tu pipeline de aprendizaje automático. El cuaderno está diseñado para evaluar toda la canalización, desde el preprocesamiento de datos hasta la realización de predicciones, garantizando que tu solución sea práctica, eficiente y precisa.


**Important Note:**
- The evaluation runs on a `single` CPU core to ensure fair resource usage comparisons.
- It assesses the pipeline’s performance as a whole, not just the final model’s predictions.

<hr style="border:2px solid gray">

**Importante:**
- La evaluación se ejecuta en un único núcleo de CPU para garantizar un uso equitativo de los recursos.
- Evalúa el rendimiento del pipeline en su conjunto, no sólo las predicciones del modelo final.

---

In [ ]:
### 1. What the Evaluation Measures / Qué mide la evaluación

The notebook evaluates your pipeline on several metrics:

#### **1.	Accuracy and Model Performance:**

- **Accuracy**: How many predictions were correct.
- **F1 Score**: A balanced measure of precision and recall, especially for imbalanced datasets.
- **Confusion Matrix**: A detailed breakdown of your model’s predictions across the following classes:
    - `Priority`: The most important class, representing clear and usable images. Misclassifying these has the greatest negative impact.
	- `Noisy` & `Blurry`: Impure images that could potentially be recovered through preprocessing. Errors here are less critical but still significant.
	- `Corrupt` & `Missing_Data`: The least important classes, representing images with severe issues or no usable data. Misclassifications here have the smallest impact.

#### **2. Evaluation Time:**

- **What we measure**: How long your pipeline takes to preprocess the data and make predictions.
- **Single CPU Core**: The evaluation runs on a `single` CPU core, not just for fairness, but to simulate a resource-constrained environment where computational resources are limited.
- **Why it matters**: Faster pipelines are more practical and scalable, especially in environments with strict performance or resource limitations.

#### **3. Memory Usage:**

- **Peak Memory Usage:** Maximum memory used during pipeline execution.
- **Average CPU Usage:** Percentage of a single core used throughout the process.
- **Why it matters:** CubeSat will use a processor with limited memory, and the memory utilization must fit within available memory.

#### **4. Algorithm Code Size:**

- The total size of your pipeline, including the serialized model and preprocessing function, measured in megabytes (MB).
- Algorithm code must fit within a limited available memory

 <hr style="border:2px solid gray">


#### **1.	Precisión y rendimiento del modelo:** 

- **Precisión**: Cuántas predicciones fueron correctas.
- **Puntuación F1**: Una medida equilibrada de precisión y recuperación, especialmente para conjuntos de datos desequilibrados.
- **Matriz de confusión**: Un desglose detallado de las predicciones de su modelo en las siguientes clases:
    - `Priority`: La clase más importante, que representa imágenes claras y utilizables. Las clasificaciones erróneas tienen el mayor impacto negativo.
	- `Noisy` & `Blurry`: Imágenes impuras que podrían recuperarse mediante preprocesamiento. En este caso, los errores son menos graves, pero no por ello menos importantes.
	- `Corrupt` & `Missing_Data`: Las clases menos importantes, que representan imágenes con problemas graves o sin datos utilizables. Los errores de clasificación tienen aquí el menor impacto.


#### **2. Tiempo de evaluación:**

- **Qué medimos**: Cuánto tarda tu pipeline en preprocesar los datos y hacer predicciones.
- **Un solo núcleo de CPU**: La evaluación se ejecuta en un **único** núcleo de CPU, no sólo por motivos de equidad, sino también para simular un entorno con recursos limitados.
- **Por qué es importante**: Los canales más rápidos son más prácticos y escalables, especialmente en entornos con limitaciones estrictas de rendimiento o recursos.


#### **3. Utilización de la memoria:**

- **Uso máximo de memoria:** Memoria máxima utilizada durante la ejecución del pipeline.
- **Uso medio de CPU:** Porcentaje de un solo núcleo utilizado durante todo el proceso.
- **Por qué es importante:** CubeSat utilizará un procesador con memoria limitada, y la utilización de la memoria debe ajustarse a la memoria disponible.

#### **4. Algoritmo Tamaño del código:**

- El tamaño total de su pipeline, incluido el modelo serializado y la función de preprocesamiento, medido en megabytes (MB).
- El código del algoritmo debe caber en una memoria disponible limitada.

---

#### Import test data / Importar datos de prueba 

In [ ]:
import numpy as np
# first let us load the testing data
test_images = np.load('data/test_images.npy')      # Load image test data
test_labels = np.load('data/test_labels.npy')      # Load label test data

**Note:** You can also use this notebook to evaluate trained models on the validation set.
    
 <hr style="border:2px solid gray">


**Nota:** También puede utilizar este Jupyter notebook para evaluar los modelos entrenados en el conjunto de validación.

#### Import Evaluation function / Importar la función de evaluación

In [9]:
from source.evaluate import evaluate_pipeline 
# A built-in function to evaluate a given ML pipeline by preprocessing, predicting, and calculating performance metrics.

**Inputs:**
  
- **model**: The trained machine learning model to evaluate.
- **X_test_raw**: Raw test data that needs to be preprocessed before evaluation.
- **y_test**: True labels corresponding to the test data for performance comparison.
- **preprocessing_fn**: A function used to preprocess the raw test data.
    
**Outputs:**

- **metrics**: A dictionary containing various evaluation metrics like accuracy, F1 score, evaluation time, memory usage, CPU usage, and algorithm code size.
- **confusion matrix**


 <hr style="border:2px solid gray">

**Entradas:**
  
- `model`: El modelo de aprendizaje automático entrenado para evaluar.
- `X_test_raw`: Datos de prueba sin procesar que necesitan ser preprocesados antes de la evaluación.
- `y_test`: Etiquetas verdaderas correspondientes a los datos de prueba para comparar el rendimiento.
- `preprocessing_fn`: Función utilizada para preprocesar los datos de prueba sin procesar.
    
**Resultados:**

- `metrics`: Un diccionario que contiene varias métricas de evaluación como precisión, puntuación F1, tiempo de evaluación, uso de memoria, uso de CPU y tamaño del código del algoritmo.
- matriz de confusión



---

### 2. ML: Evaluation (3rd notebook) / Evaluación del aprendizaje automático (Jupyter notebook 3)

#### Preprocessing (Testing data) / Preprocesamiento (datos de prueba)


Applying consistent preprocessing to both training and testing data ensures accurate predictions, prevents mismatches, and improves model evaluation. Combining all steps into one function enhances consistency, tracks execution time, and optimizes resource usage.


 <hr style="border:2px solid gray">

La aplicación de un preprocesamiento coherente tanto a los datos de entrenamiento como a los de prueba garantiza predicciones precisas, evita desajustes y mejora la evaluación de los modelos. Combinar todos los pasos en una sola función mejora la coherencia, controla el tiempo de ejecución y optimiza el uso de recursos.



In [ ]:
def preprocessing_fn_ML(X): # We apply the same pre-processing steps implemented in Notebook 3.
    from skimage.color import rgb2gray
    from skimage.transform import resize
    
    # Normalize the data to [0, 1]
    X_pre = X.astype('float32') / 255.0
    
    # Convert to grayscale
    X_pre = np.array([rgb2gray(image) for image in X_pre])
    
    # Resize images to 64x64 pixels
    X_pre = np.array([resize(image, (64, 64), anti_aliasing=True) for image in X_pre])
    
    # Flatten the images
    num_samples = X_pre.shape[0]
    X_pre = X_pre.reshape(num_samples, -1)
    
    return X_pre

In [ ]:
#### Import ML model / Importar modelo de aprendizaje automático

In [ ]:
import pickle
# Load the ml model from the 3rd notebook
with open('models/sgd_model.pkl', 'rb') as file:
    sgd_model = pickle.load(file)

#### ML Evaluation / Evaluación de modelo de aprendizaje automático

In [ ]:
# Assuming you have:
# - A trained model named like 'lr_model'
# - Raw test data 'X_test_raw'
# - True labels 'y_test'
# - All pre-processing methods gathered in one function


# Evaluate the pipeline
metrics = evaluate_pipeline(sgd_model, test_images, test_labels, preprocessing_fn_ML)

---

### 3. CubeSatNet_CNN Evaluation / Evaluación de CubeSatNet_CNN

#### preprocessing / preprocesamiento

In [ ]:
def preprocessing_fn_CNN(X):  # we did not use any preprocessing in notebook 4

    return X

#### Import CNN model / Importar modelo CNN

In [ ]:
from keras.utils import to_categorical
test_labels = to_categorical(test_labels, num_classes=5)

# Load the CNN model from the 4th notebook
with open('models/cnn_model.pkl', 'rb') as file:
    cnn_model = pickle.load(file)

#### Evaluate the CNN pipeline /  Evaluar el CNN

In [ ]:
# Evaluate the pipeline
metrics = evaluate_pipeline(cnn_model, test_images, test_labels, preprocessing_fn_CNN)


##### **⚠️ Freeing up Space** / **⚠️ Liberar espacio**

In [ ]:
import gc

# Remove the data from memory
del sgd_model, cnn_model, test_images, test_labels

# Force garbage collection to free up memory
gc.collect()

# Clear the input/output cache
print("Data and models removed from memory.")

---

**Note:**

1. You can also use the `evaluate_pipeline` function to evaluate your validation set data.
2. Please do not modify the `evaluate_pipeline` function.

Best of luck with the hackathon challenge! 🎉 We look forward to seeing your final results and pipelines! 🚀


 <hr style="border:2px solid gray">


**Nota:**

1. También puede utilizar la función `evaluate_pipeline` para evaluar los datos de su conjunto de validación.
2. Por favor, no modifique la función `evaluate_pipeline`.

¡Mucha suerte con el reto del hackathon! 🎉 ¡Estamos deseando ver tus resultados finales y pipelines! 🚀